In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [0]:
!pip install pyspark



     |████████████████████████████████| 217.8MB 61kB/s 
     |████████████████████████████████| 204kB 47.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=143621fcc47da2f3b584b63362883cd47ef1565f762b065ab7d196c3147b9e2e
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"12XxEzxgvuPbrDu4cIsaFVQApBq6pbdlS"})   # replace the id with id of file you want to access
downloaded.GetContentFile('nf_subsample.csv')        # replace the file name with your file

In [0]:
import sys
import numpy as np
import csv
from pyspark import SparkContext

class strata:
    def __init__(self, n):
        self.strata = list(range(n))
    def __iter__(self):
        return self
    def next(self):
        for i in list(range(len(self.strata))):
            if self.strata[i] + 1 == len(self.strata):
                self.strata[i] = 0
            else:
                self.strata[i] += 1
        return self.strata

def parseInput(line):
    """
    Parse each row into (mid, uid, rate) format.
    """
    tmp = line.split(",")
    return (int(tmp[0]), int(tmp[1]), float(tmp[2]))

def divideSets(lst):
    """
    Divide a list into numWorker pieces.
    """
    ret = []
    for i in list(range(numWorker)):
        ret.append(set())
    index = 0
    while index < len(lst):
        ret[index%numWorker].add(lst[index])
        index += 1
    return ret

def buildInvertedIndex(listOfSets):
    """
    Build inverted index for mid vs block id.
    Input:
        list of [ [mid1, mid2, ...], [mid11, mid12, ...], ... ]
    Output:
        { mid1:0, mid2:0, ..., mid11:1, mid12:1, ... }
    """
    ret = dict()
    for i in list(range(len(listOfSets))):
        for elem in listOfSets[i]:
            ret[elem] = i
    return ret

def writeWandH():
    """
    Write the final output for w.csv, h.csv
    """
    count=0
    with open(outputWpath, 'w') as wFile:
        writer = csv.writer(wFile)
        for mid in sorted(wk):
            count = count + 1
            writer.writerow(wk[mid])
            
    with open(outputHpath, 'w') as hFile:
        writer = csv.writer(hFile)
        index = sorted(hk)
        for i in list(range(numFactor)):
            writer.writerow([hk[x][i] for x in index])

def main():
    rawData = sc.textFile(inputVpath, numWorker).map(parseInput)
    movieIndex = sorted(rawData.map(lambda x : x[0]).distinct().collect())
    userIndex  = sorted(rawData.map(lambda x : x[1]).distinct().collect())
    print('movie index', movieIndex)
    print('movie len: ', len(movieIndex))
    print('user index', userIndex)
    print('usr len', len(userIndex))
    pMovieIndex = divideSets(movieIndex)
    pUserIndex  = divideSets(userIndex)
    print('pMovieIndex[0]: ', pMovieIndex[0])
    print('pUserIndex[0]: ',pUserIndex[0])
    invertedMovieIndex = buildInvertedIndex(pMovieIndex)
    print('invertedMovieIndex[0]: ',invertedMovieIndex)
    rawData = rawData.map(lambda x : (invertedMovieIndex[x[0]], x))
    rawData.foreach(print)
    rawData = rawData.partitionBy(numWorker).map(lambda x : x[1])
    rawData.cache()
    print(rawData.collect())
    totalSelect = rawData.count()
    print(pUserIndex[0])
    bUserIndex = sc.broadcast(pUserIndex)
    print('lllll : ', bUserIndex.value[0])


    nI = rawData.map(lambda x : (x[0], 1)).reduceByKey(lambda a, b: a + b).collect()
    nJ = rawData.map(lambda x : (x[1], 1)).reduceByKey(lambda a, b: a + b).collect()
    bNI = sc.broadcast(dict(nI))
    bNJ = sc.broadcast(dict(nJ))
    bLambdaV = sc.broadcast(lambdaV)
    bMBetaV = sc.broadcast(-betaV)
    #### Read data and broadcast done. ####

    def showStatus(id, iterator):
        """
        Show status of each worker in current strata.
        """
        ret = []
        wGrad = dict()
        hGrad = dict()
        for elem in iterator:
            print('coool: ', curStrata[id])
            if elem[1] in bUserIndex.value[curStrata[id]]:  #### only consider those data in this strata.
                mid = elem[0]
                uid = elem[1]
                rating = elem[2]
                predict = wk[mid].dot(hk[uid])
                error = rating - predict
                ret.append((mid, uid, rating, predict, error, numI.value[mid], numJ.value[uid]))
        yield (id, ret)

    def calRMSE(elem):
        """
        Calculate RMSE for debug.
        """
        RMSE = 0.0
        mid = elem[0]
        uid = elem[1]
        rating = elem[2]
        predict = wk[mid].dot(hk[uid])
        RMSE += (rating - predict)**2
        return RMSE

    def showRMSE(elem):
        """
        Show info about RMSE for debug.
        """
        RMSE = 0.0
        mid = elem[0]
        uid = elem[1]
        rating = elem[2]
        predict = wk[mid].dot(hk[uid])
        RMSE += (rating - predict)**2
        return (mid, uid, rating, predict, rating - predict)

    def updateBlock(id, iterator):
        """
        Calculate gradient in worker_id.
        """
        wGrad = dict()
        hGrad = dict()
        mbi = 0
        for elem in iterator:
            if elem[1] in bUserIndex.value[curStrata[id]]:  #### only consider those data in this strata.
                mid = elem[0]
                uid = elem[1]
                rating = elem[2]
                if mid not in wGrad:
                    wGrad[mid] = wk[mid]
                if uid not in hGrad:
                    hGrad[uid] = hk[uid]
                epsilon = 2*(100.0 + mbi + totalN) ** (bMBetaV.value)
                error = wGrad[mid].dot(hGrad[uid]) - rating
                tmpw = error*hGrad[uid] + (bLambdaV.value/bNI.value[mid])*wGrad[mid]
                tmph = error*wGrad[mid] + (bLambdaV.value/bNJ.value[uid])*hGrad[uid]
                wGrad[mid] -= epsilon*tmpw
                hGrad[uid] -= epsilon*tmph
                mbi += 1
        yield (id, [wGrad, hGrad, mbi])

    #### Ininialize W and H matrix. ####
    global wk, hk
    wk = dict()
    hk = dict()
    for mid in movieIndex:
        wk[mid] = np.random.random(numFactor)
    for uid in userIndex:
        hk[uid] = np.random.random(numFactor)

    ### Ininialize strate array.
    myStrata = strata(numWorker)
    print('.... : ' ,type(wk))
    
    totalN = 0
    for t in list(range(numIteration)):
        curStrata = myStrata.next()
        #print(curStrata)
    #    curRMSE = rawData.map(calRMSE).reduce(lambda a,b : a+b)
    #    print "total RMSE = ", curRMSE
        grads = rawData.mapPartitionsWithIndex(updateBlock).collectAsMap()
        for key, value in grads.items():
            for k, v in value[0].items():
                wk[k] = v
            for k, v in value[1].items():
                hk[k] = v
            totalN += value[2]
        #print "t = {}, totalN = {}".format(t, totalN)
        curRMSE = rawData.map(calRMSE).reduce(lambda a,b : a+b) / totalSelect
        #print('cooolppppppppppppppppppppppppppppppppppp')
        #print >> sys.stderr, "total RMSE = ", curRMSE
    writeWandH()


In [0]:
if __name__ == "__main__":
    np.random.seed(121)

    numFactor = 10
    numWorker = 10
    numIteration = 30
    betaV = 0.6
    lambdaV = 0.3
    inputVpath = 'nf_subsample.csv'
    outputWpath = 'w.csv'
    outputHpath = 'h.csv'

    ### Initialize SparkContext
    setting = 'local[%d]' % numWorker
    sc = SparkContext(setting, 'pyspark')
    main()

movie index [6, 7, 8, 10, 25, 33, 42, 59, 79, 83, 87, 94, 97, 116, 126, 130, 131, 133, 134, 142, 149, 158, 164, 168, 169, 177, 178, 183, 188, 192, 195, 199, 201, 215, 231, 242, 247, 248, 250, 261, 265, 266, 267, 268, 283, 291, 296, 298, 299, 301, 302, 304, 305, 307, 308, 310, 312, 314, 330, 331, 333, 352, 358, 363, 368, 369, 383, 384, 385, 392, 413, 416, 424, 437, 439, 440, 442, 453, 462, 466, 470, 471, 477, 478, 479, 481, 485, 490, 491, 492, 495, 508, 517, 527, 529, 536, 544, 546, 547, 550, 561, 576, 578, 585, 592, 596, 602, 609, 614, 616, 623, 624, 633, 657, 660, 663, 664, 665, 682, 684, 685, 688, 695, 711, 719, 734, 735, 739, 742, 744, 748, 750, 756, 766, 767, 769, 781, 784, 785, 787, 788, 793, 798, 815, 825, 829, 834, 840, 844, 857, 870, 873, 877, 906, 909, 911, 915, 920, 930, 933, 939, 944, 952, 955, 962, 967, 968, 979, 981, 989, 997, 998, 1007, 1020, 1024, 1034, 1038, 1044, 1047, 1059, 1067, 1069, 1070, 1079, 1082, 1086, 1088, 1090, 1097, 1109, 1117, 1135, 1146, 1179, 1186, 1188,

In [0]:
sc.stop()

In [0]:
R = np.zeros((338,839));
with open(inputVpath, 'r') as file:
    array = file.readlines()
    array = [row.split(',') for row in array]
    for p in range(len(array)):
      print(int(array[p][0])," ",int(array[p][1]))
      R[int(array[p][0])][int(array[p][1])] = float(array[p][2])
          #if(k!=2):
           # array[p][k] = int(array[p][k])
          #else:
           # array[p][k] = float(array[p][k])

915   1


IndexError: ignored

In [0]:
with open(outputHpath, 'r') as file:
    harray = file.readlines()
    harray = [row.split(',') for row in harray]
    for p in range(len(harray)):
      for k in range(len(harray[0])):
        harray[p][k] = float(harray[p][k])
    print(len(harray)," ",len(harray[0]))

10   839


In [0]:
with open(outputWpath, 'r') as file:
    warray = file.readlines()
    warray = [row.split(',') for row in warray]
    for p in range(len(warray)):
      for k in range(len(warray[0])):
        warray[p][k] = float(warray[p][k])
    print(warray)
    print(len(warray)," ",len(warray[0]))

[[0.2637039114278993, 0.3960399840638832, 0.3570190914024499, 0.22605508836443367, 0.8501207983091768, 0.48745470126734636, 0.7059053057889332, 0.7916689776419498, 0.431579049653556, 1.156660248820769], [1.0821537994681707, 0.5399329124850633, 0.24000221784169157, 1.0880282931396896, 0.5726112589420983, 0.8128571162590859, 0.4422863068639098, 1.0571192197711716, 0.5375751691649634, 0.759927037756215], [0.32701067938397715, 0.28711044098548144, 0.09204197290028963, 0.6214761096954711, 0.8358252697755293, 0.7534472628185219, 0.9453480622441707, 0.8211795517726758, 0.713119279662545, 0.7606586577530461], [0.4967703557353815, 0.5775627598959101, 0.6622801661967235, 0.8721626012481095, 0.20095342932078306, 0.061396137805645325, 0.47031985641959073, 0.5337128695148162, 0.7456639154959329, 0.8558976102127563], [0.7870862161768674, 0.16472228634351185, 0.6682888828371143, 0.6916351208378873, 0.1250644974025844, 0.20103417908329352, 0.10332698279595522, 0.2865951440928601, 0.49081492903057394, 

In [0]:
import numpy as np
predicted_ratings = np.matmul(warray, harray)

In [0]:
print(predicted_ratings)   # m x n = 338 x 839 = movies x users
print(predicted_ratings.shape)

[[2.55632133 2.85331721 2.24974137 ... 2.65688755 2.79015254 2.86559689]
 [2.98205761 3.63585304 3.01379778 ... 2.99870936 4.08112294 4.27112804]
 [2.59206509 3.26593826 2.05553209 ... 2.42562667 3.24530625 3.21043139]
 ...
 [3.24454554 3.24514791 2.8348812  ... 3.07155186 3.71182673 4.25266684]
 [2.09479798 1.88862922 2.2401157  ... 2.20718087 2.60080507 2.63795066]
 [2.89964345 3.38041383 3.21755168 ... 3.03189879 3.73050311 3.90119278]]
(338, 839)


In [0]:
print(predicted_ratings)   # m x n = 338 x 839 = movies x users
print(predicted_ratings.shape)
print(predicted_ratings[np.where(predicted_ratings[:,0]==915)][np.where(predicted_ratings[:,1]==1)])

[[2.55632133 2.85331721 2.24974137 ... 2.65688755 2.79015254 2.86559689]
 [2.98205761 3.63585304 3.01379778 ... 2.99870936 4.08112294 4.27112804]
 [2.59206509 3.26593826 2.05553209 ... 2.42562667 3.24530625 3.21043139]
 ...
 [3.24454554 3.24514791 2.8348812  ... 3.07155186 3.71182673 4.25266684]
 [2.09479798 1.88862922 2.2401157  ... 2.20718087 2.60080507 2.63795066]
 [2.89964345 3.38041383 3.21755168 ... 3.03189879 3.73050311 3.90119278]]
(338, 839)
[]
